In [2]:
#DATASET
import pandas as pd
url = 'https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv'
df = pd.read_csv(url, sep='\t')
df.columns = ['label', 'message']
df.head()
df.shape

(5571, 2)

In [ ]:
#PREPROCESSING
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 3: Define a function for text preprocessing
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return text

# Step 4: Apply preprocessing to all messages
df['clean_message'] = df['message'].apply(clean_text)

# Step 5: Apply TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_message'])  # Numerical feature matrix


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# Uncomment classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def encode_labels(df):
    # Return the encoded labels
    return df['label'].map({'ham': 0, 'spam': 1}).fillna(-1)

def train_models(X, y):
    # Convert the boolean pandas Series to a NumPy array for indexing the sparse matrix
    y_filter = (y != -1).values
    X_filtered = X[y_filter]
    y_filtered = y[y_filter]

    # Ensure y_filtered is also a numpy array or similar for train_test_split
    if isinstance(y_filtered, pd.Series):
        y_filtered = y_filtered.values

    X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Naive Bayes": MultinomialNB(),
        "Random Forest": RandomForestClassifier(),
        "SVM": SVC(probability=True)
    }

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(f"\n{name} Accuracy: {accuracy_score(y_test, y_pred):.2f}")
        # Ensure y_test and y_pred are compatible types for classification_report
        print(classification_report(y_test, y_pred))

# Apply the encoding and update the 'label' column before calling train_models
df['label'] = encode_labels(df)

# Call train_models with the updated label column
train_models(X, df['label'])


Logistic Regression Accuracy: 0.93
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       955
           1       0.99      0.54      0.70       160

    accuracy                           0.93      1115
   macro avg       0.96      0.77      0.83      1115
weighted avg       0.94      0.93      0.93      1115


Naive Bayes Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       955
           1       1.00      0.67      0.80       160

    accuracy                           0.95      1115
   macro avg       0.97      0.83      0.89      1115
weighted avg       0.95      0.95      0.95      1115


Random Forest Accuracy: 0.97
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       955
           1       1.00      0.78      0.87       160

    accuracy                           0.97      1115
   macro avg       0.98      0.89 